<a href="https://colab.research.google.com/github/noahlearner/autosuggest/blob/main/Google_Suggestions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
#load libraries
import pandas as pd 
import requests
from bs4 import BeautifulSoup
from string import ascii_lowercase
import json 

In [ ]:
#Get input from users
#@markdown ### Enter your Keyword
seed_kw = "plumber" #@param {type:"string"}
#@markdown ### Enter the country (Search Volume is assigned at the country level)
country = "US" #@param {type:"string"}

In [ ]:
#Dataset with Table name variable
import pandas_gbq

#@markdown ### Enter your BigQuery Project ID - Leave As crawling-284315 unless you want to put data elsewhere.
#BQproject name variable
project='crawling-284315' #@param {type:"string"}
# pandas-gbq method to load data
# append data if data table exists in BQ project 
# set chunk size of records to be inserted
#@markdown ### Enter your BigQuery dataset. For TO team, leave the dataset as "autosuggest".
dataset='autosuggest' #@param {type:"string"}

#@markdown ### Enter your BigQuery table.  Replace 'plumber' with website name.
table='brett' #@param {type:"string"}

#@markdown ---
bqTable = dataset + '.'+ table


In [ ]:
#@title
#convert seed keyword to list
keywords = [seed_kw]
#create aditionnal seeed by appending a-z & 0-9 to it
for c in ascii_lowercase:
  for c2 in ascii_lowercase:
    keywords.append(seed_kw+' '+c+c2)
    keywords.append(c+c2+' '+seed_kw+' '+c+c2)
for i in range(0,10):
  keywords.append(seed_kw+' '+str(i))

#gett all suggestions from Google
sugg_all = []
i=1

for kw in keywords:
  r = requests.get('http://suggestqueries.google.com/complete/search?output=toolbar&hl={}&q={}'.format(country,kw))
  soup = BeautifulSoup(r.content, 'html.parser')
  sugg = [sugg['data'] for sugg in soup.find_all('suggestion')]
  sugg_all.extend(sugg)

In [ ]:
#@title
#remove duplicated
sugg_all = pd.Series(sugg_all).drop_duplicates(keep='first')
if len(sugg_all)==0:
  print('There are no suggestion. The script can\'t work :(')
  
#get search volume data 
#prepare keywords for encoding
data = sugg_all.str.replace(' ','%20').unique()
#divide kws into chunks of kws
chunks = [data[x:x+25] for x in range(0, len(data), 25)]

#create dataframe to receive data from API
results = pd.DataFrame(columns=['keyword','volume'])

#get data 
for chunk in chunks:
  url = (
      'https://db2.keywordsur.fr/keyword_surfer_keywords?country={}&keywords=[%22'.format(country)+
      '%22,%22'.join(chunk)+
      '%22]'
  )

  r = requests.get(url)
  try:
    data = json.loads(r.text)
  except:
    continue

  for key in data.keys():
    results.loc[len(results)] = [key,data[key]['search_volume']]

In [ ]:
#@title
results = (
    sugg_all
    .to_frame()
    .rename({0:'keyword'},axis=1)
    .merge(results,on='keyword',how='left')
    .fillna(0)
)
filtered_results = results.query('volume >0')
filtered_results.sort_values(by='volume',ascending=False).to_csv('data.csv',index=False)

In [ ]:
#@title
%load_ext google.colab.data_table
filtered_results

,keyword,volume
0,plumbers near me,450000
1,plumber,368000
2,plumber salary,18100
3,plumbers putty,18100
4,plumbers tape,18100
...,...,...
4037,plumber 911,1000
4041,plumber 90045,10
4042,plumber 90066,10
4044,plumber 92130,10


In [ ]:
#@title

filtered_results.to_gbq(bqTable, project,chunksize=10000, if_exists='append')
print('success!!!')